In [4]:
import numpy as np
import pandas as pd

from tqdm import tqdm 
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax

from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score
)

MODEL_NAME = "cardiffnlp/bertweet-base-sentiment"
MODEL_PATH = f"../models/{MODEL_NAME}"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
# model.save_pretrained(MODEL_PATH)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    local_files_only=True
)

config.json: 100%|██████████| 837/837 [00:00<00:00, 4.10MB/s]
vocab.txt: 100%|██████████| 843k/843k [00:00<00:00, 1.02MB/s]
bpe.codes: 100%|██████████| 1.08M/1.08M [00:01<00:00, 920kB/s]
added_tokens.json: 100%|██████████| 17.0/17.0 [00:00<00:00, 26.9kB/s]
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
pytorch_model.bin: 100%|██████████| 540M/540M [00:53<00:00, 10.0MB/s] 


In [6]:
def preprocess(text):
    new_text = []
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        # t = '' if t.startswith(('@', 'http')) and len(t) > 1 else t
        new_text.append(t)
    return " ".join(new_text)

def infer(text):
    labels = ['negative', 'neutral', 'positive']
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    output = ranking[0]
    label = labels[output]
    return text, label

In [7]:
def evaluate(predictions, true_labels):
    cls_report = classification_report(
        true_labels, 
        predictions
    )
    cnf_matrix = confusion_matrix(
        true_labels, 
        predictions
    )
    accuracy = accuracy_score(true_labels, predictions)

    print("Classification Report:\n", cls_report)
    print("Confusion Matrix:\n", cnf_matrix)
    print("Accuracy: ", accuracy)


def test_batch(csv_path):
    df = pd.read_csv(csv_path)
    data = df['text'].tolist()
    true_labels = df['expected_sentiment'].tolist()
    predictions = []

    for i in tqdm(range(len(data)), desc="Processing"):
        text = data[i]
        true_label = true_labels[i]
        processed_text, predicted_label = infer(text)
        predictions.append(predicted_label)

    evaluate(predictions, true_labels)

In [8]:
csv_path = '../data/sentiment_test_cases.csv'
test_batch(csv_path)

Processing:   0%|          | 0/498 [00:00<?, ?it/s]

Processing: 100%|██████████| 498/498 [00:19<00:00, 25.63it/s]

Classification Report:
               precision    recall  f1-score   support

    negative       0.92      0.82      0.87       177
     neutral       0.78      0.81      0.80       139
    positive       0.83      0.88      0.85       182

    accuracy                           0.84       498
   macro avg       0.84      0.84      0.84       498
weighted avg       0.85      0.84      0.84       498

Confusion Matrix:
 [[145  16  16]
 [  8 113  18]
 [  5  16 161]]
Accuracy:  0.8413654618473896
